<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Introduction-with-conclusions" data-toc-modified-id="Introduction-with-conclusions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction with conclusions</a></span></li><li><span><a href="#Problems-encountered-in-your-map" data-toc-modified-id="Problems-encountered-in-your-map-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Problems encountered in your map</a></span><ul class="toc-item"><li><span><a href="#SYM_UL-for-every-street" data-toc-modified-id="SYM_UL-for-every-street-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>SYM_UL for every street</a></span></li><li><span><a href="#Data-importer-based-on-the-sqlite3-python-library---&quot;csv_to_sql.py&quot;" data-toc-modified-id="Data-importer-based-on-the-sqlite3-python-library---&quot;csv_to_sql.py&quot;-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data importer based on the sqlite3 python library - "<code>csv_to_sql.py</code>"</a></span></li><li><span><a href="#&quot;version&quot;-tag-as-the-integer---versionchecker.py" data-toc-modified-id="&quot;version&quot;-tag-as-the-integer---versionchecker.py-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>"version" tag as the integer - <code>versionchecker.py</code></a></span></li></ul></li><li><span><a href="#Overview-of-the-Data" data-toc-modified-id="Overview-of-the-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Overview of the Data</a></span><ul class="toc-item"><li><span><a href="#Counting-tags" data-toc-modified-id="Counting-tags-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Counting tags</a></span></li><li><span><a href="#Counting-users" data-toc-modified-id="Counting-users-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Counting users</a></span></li><li><span><a href="#Counting-number-of-kindergardens-/-schools" data-toc-modified-id="Counting-number-of-kindergardens-/-schools-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Counting number of kindergardens / schools</a></span></li><li><span><a href="#Time-of-actualization" data-toc-modified-id="Time-of-actualization-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Time of actualization</a></span></li></ul></li><li><span><a href="#Literature" data-toc-modified-id="Literature-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Literature</a></span></li></ul></div>

In [7]:
import sys

sys.path.append('/data')


import xml.etree.ElementTree as ET
import os
import re
import pyprind
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt


from scripts import osmcsv
from collections import defaultdict
from __future__ import division
from scripts.tablecreator import table_creator as table_creator
%matplotlib inline

# Setting file
wroclaw = '../wroclaw'

# Introduction with conclusions

Wrocław (Wroclaw) is a city with in Poland where I live. It is one of the fastest growing cities in Poland economically as well as in the size aspect: city authorities are increasing expenses on the infrastructure  annualy and therefore Wrocław have a lot of building or modernization sites. This concern also streets - as the Wroclaw citizen should be up-to-date with the information on the streets currently being renovated and possible traffics that could be met  on the way to school or to work. 

<img src="./data/wroclaw.jpg">
<center>[Source: Vogue, https://www.vogue.com/article/wroclaw-poland-travel-guide-under-the-radar]</center>

<img src="https://map.viamichelin.com/map/carte?map=viamichelin&z=10&lat=51.11049&lon=17.02582&width=550&height=382&format=png&version=latest&layer=background&debug_pattern=.*">

<center>[Source: ViaMichelin]</center>

Problems encountered and solution thereto:

1. In the Polish OSM we have another tag attribute ("SYM_UL"), a unique number assigned to the each street in the particular. I have found that in my OSM there are three streets without such information, so I filled this lack and create second table mapping street names and separate codes. 
2. Due to the problem with importing data from csv files (nodes.csv, ways.csv), I have created an own importer based on the sqlite3 python library. 
2. Due to the ensure consistency between Python schema and SQL schema I have pointed "version" tag as the integer in Python schema. 

Questions asked:

1. As the open source fan, I wonder how many users participate in the creation of the map. 
2. Due to the application OSM as the GPS backend I would like to see whether it is not outdated.
2. Due to having kids and considering moving from Wroclaw - how many kindedrgardens my child may go to in the future. 

#  Problems encountered in your map

## SYM_UL for every street

We could see that most frequent tags in the nodes tags table are following:

In [8]:
%load_ext sql

In [9]:
%sql sqlite:///data/test.db

'Connected: None@data/test.db'

In [10]:
%%sql 
SELECT key, COUNT (key) 
  FROM  nodes_tags 
 GROUP BY key 
 ORDER by count(key) 
  DESC LIMIT 10;

Done.


key,COUNT (key)
height,23177
OBJECTID_height_lidar,11469
street,4765
housenumber,4740
city,4698
postcode,4546
street:sym_ul,4266
country,3892
name,2886
highway,1911


From the above we could see that  there are 4765 street tags, however "sym_ul" have 4266 positions. 

What is interesting in the Polish section there is a unique key addded to each street ("street: sym_ul"). As you are probably  an English speaking reader you may google translate this thread: https://forum.openstreetmap.org/viewtopic.php?id=59111 or here https://wiki.openstreetmap.org/wiki/Talk:Pl:Importy/Adresy. 

Unfortunately this tag is not documented in English. Sym_ul is the id added by users on basis of the registry by the Polish National Statistical Office. 

Let's check how many unique streets we have in the nodes_tags table and how many unique "sym_ul" tags we have therein too. 

In [7]:
%%sql
CREATE VIEW streets AS
     SELECT value 
       FROM nodes_tags
      WHERE key = "street"
      UNION SELECT value
       FROM ways_tags
      WHERE key = "street"
      GROUP BY value);

(sqlite3.OperationalError) near ")": syntax error [SQL: 'CREATE VIEW streets AS\n     SELECT value \n       FROM nodes_tags\n      WHERE key = "street"\n      UNION SELECT value\n       FROM ways_tags\n      WHERE key = "street"\n      GROUP BY value);']


In [8]:
%%sql
SELECT COUNT(*) as "Total number fo streets"
FROM 
    (SELECT value 
       FROM nodes_tags
      WHERE key = "street"
      UNION SELECT value
       FROM ways_tags
      WHERE key = "street"
      GROUP BY value);

Done.


Total number fo streets
292


In [9]:
%%sql
SELECT COUNT(*) AS "Total number of sym_ul tags" 
 FROM 
    (SELECT value 
       FROM nodes_tags
      WHERE key = "street:sym_ul"
      UNION SELECT value
       FROM ways_tags
      WHERE key = "street:sym_ul"
      GROUP BY value);



Done.


Total number of sym_ul tags
290


Then - after making investigation the entire dataset (so also the ways_tags with the streetmapping file we could find that the following streets have got no "sym_ul" tags: 
1. Grabarska
2. Adama Mickiewicza
3. Na Niskich Łąkach

I have changed it basing on the official base (http://eteryt.stat.gov.pl/eTeryt/rejestr_teryt/udostepnianie_danych/baza_teryt/uzytkownicy_indywidualni/wyszukiwanie/wyszukiwanie.aspx?contrast=default) 

I have also created the table which enable the database user to assign street codes with the street name ("```streetmapping.csv```") with the following SQL equivalent: 

## Data importer based on the sqlite3 python library - "```csv_to_sql.py```"

After importing data to csv I have found some issues:

```nodes.csv:253769: INSERT failed: UNIQUE constraint failed: nodes.id```



We will check it manually during uploading each node separately from node.csv into the sqlite3. Please find below an exemplary chunk fo the code: 

In [183]:
for num, i in enumerate(range(nodes.shape[0])):
    id_key, lat, lon, user, uid, version, changeset, timestamp = nodes.iloc[i]

    c.execute('''INSERT INTO nodes 
                (id, lat, lon, user, uid, version, changeset, timestamp)
        VALUES ({},{},{},'{}',{},{},{},'{}')'''.format(id_key, 
                                                   lat, 
                                                   lon, 
                                                   user, 
                                                   uid, 
                                                   version, 
                                                   changeset, 
                                                   timestamp)
         )
    
conn.commit()
conn.close()

After applying the script "```csv_to_sql.py```" I have not encountered any errors. __During merging data from one source to another  (in our case from csv to sql) some errors can occur that could be  ommitted by another API. Here we have ommitted this error through applying the python script what spare our time __

##  "version" tag as the integer - ```versionchecker.py```

From the original schema we have:

In [ ]:
table_creator = """CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);

In the pythonic way

Name | Type of data SQL | Type of data python 
:-:|-|-
id| integer | int [primary key]
lat | REAL | float
lon | REAL | float
user | TEXT | int
uid | INTEGER | int
version | INTEGER | int
changeset | INTEGER | int
timestamp | TEXT | string


In the file schema we have however:

```'version': {'required': True, 'type': 'string'},```

What is inconsistent with the integer type, let's try to convert the string to int:

In [2]:
def version_checker(filename):
    for _, elem in ET.iterparse(filename):
        if elem.tag == "node":
            try:
                int(elem.attrib["version"])
            except ValueError:
                print("There is a piece of data that cannot be converted to the integer")
                break
    print("Any errors have occured")
                
version_checker(wroclaw)

Any errors have occured


Thus, Python is able to convert it to the integer so I have updated schema with:

```'version': {'required': True, 'type': 'string'},```



# Overview of the Data

## Counting tags

In [2]:
def count_tags(filename):
    tags = defaultdict(lambda: 0)
    for i in ET.iterparse(filename):
        tags[i[1].tag] += 1
    return tags

In [3]:
tags = count_tags(wroclaw)

In [7]:
tag_summary = pd.DataFrame.from_dict(tags, orient='index')
tag_summary.columns = ["Number of elements"]
tag_summary

,Number of elements
way,39294
member,68743
osm,1
note,1
tag,214770
bounds,1
nd,360671
node,253769
meta,1
relation,1290


## Counting users

In [12]:
%%sql
CREATE VIEW users AS 
SELECT user AS user 
FROM nodes 
UNION ALL
SELECT user as user FROM ways;

SELECT user, COUNT(*) FROM users 
 GROUP BY user 
 ORDER BY count(*) 
 DESC LIMIT 10;


0 rows affected.
Done.


user,COUNT(*)
rowers2,112861
lms,74278
maraf24,70277
Ancymon,7109
psboypl,5717
Nolan Proost,2205
mrIimbsf,2176
Zmianowy,2174
marek kleciak,1948
one_half_3544,940


There are three guys that created more than 250000 elements! 

In [14]:
%%sql
SELECT count(user) as "No. of users" FROM users;


Done.


No. of users
293063


In [34]:
(112861 + 74278 + 70277)/293063

0.8783640377666235

It is 88% of the elements! The dataset should be verified due to the fact that only three users created almost 88% of the  map. 

It could influence potentially negatively on the quality of the map -  when the map is not verified by more users it could cause error to GPS system using OSM. 

## Counting number of kindergardens / schools 

As I have mentioned due to the having a kid, it is important to assure him a proper education and chances for the best learning opportunities as well as to play. Let's check number of kindergardens and schools in the neighbourhood.

In [15]:
%%sql 
SELECT * 
  FROM nodes_tags 
 WHERE key = "amenity" 
   AND value LIKE "kindergarten" 
    OR value LIKE "school"
    OR value LIKE "playground";

Done.


id,key,value,type
1168111478,amenity,kindergarten,regular
1200435506,amenity,school,regular
1200503957,amenity,kindergarten,regular
1375026875,amenity,kindergarten,regular
2723841832,amenity,school,regular
2891586837,leisure,playground,regular
2892313551,amenity,school,regular
3479947758,amenity,school,regular
3479947759,amenity,school,regular
3479947760,amenity,school,regular


This value is however too small as for the Wrocław. According to this site, the total number of schools is 

In [24]:
%sql SELECT COUNT(*) as "Total fast foods" FROM nodes_tags WHERE key = "amenity" AND value LIKE "fast_food";

Done.


Total fast foods
87


It is not good place for my kid. 

```
sqlite> select *  from nodes_tags WHERE key = "amenity" AND value LIKE "k%";
```
```
1168111478,amenity,kindergarten,regular
1200503957,amenity,kindergarten,regular
1375026875,amenity,kindergarten,regular
3834091857,amenity,kindergarten,regular
5016353990,amenity,kindergarten,regular
```

## Time of actualization

Due to many renovation and modernization of ways the map that may be used for GPS should be up to date. Let's check how  many times the user intervene in the ways tags: 

In [12]:
%%sql
SELECT strftime("%Y", timestamp) AS Year, COUNT(*) AS "Number of timestamps"
 FROM ways 
GROUP BY strftime("%Y",timestamp);

Done.


Year,Number of timestamps
2008,12
2009,8
2010,4
2011,61
2012,55
2013,92
2014,2318
2015,4789
2016,2857
2017,29051


Between 2008 and 2013 there were not many interventions made by the users, however from 2014 the base seems to be updated. The threshold is at 2017. I will demonstrate the results made in the each year since January 2016:



In [33]:
%%sql
DROP VIEW last_year;
CREATE VIEW last_year AS 
     SELECT strftime("%Y-%m", timestamp) AS "Year-Month", COUNT(*) FROM ways 
      WHERE strftime("%Y", timestamp) > "2015" 
     GROUP BY strftime("%Y-%m", timestamp);

SELECT * FROM last_year;

0 rows affected.
0 rows affected.
Done.


"strftime(""%Y-%m"", timestamp)",COUNT(*)
2016-01,82
2016-02,103
2016-03,129
2016-04,58
2016-05,207
2016-06,172
2016-07,236
2016-08,429
2016-09,165
2016-10,460


Let's demonstrate it on the graph:

In [42]:
conn = sqlite3.connect("data/test.db")
c = conn.cursor()

data = c.execute("SELECT * FROM last_year")

In [43]:
years = data.fetchall()

In [50]:
plt.rcParams["figure.figsize"] = [10,10]
last_year = pd.DataFrame({"Month": years.keys()(years), orient="index")
last_year.columns = ["Month", "Count"]
x=[i for i in range(len(last_year["Month"]))]
plt.plot(x, last_year["Count"])
plt.xticks([i for i in range(len(last_year["Month"])) if not i%3],
            [i for num, i in enumerate(last_year["Month"]) if not num%3])    
plt.xlabel("Year - month")
plt.ylabel("Number of updates")
plt.title("Number of updates of Wroclaw's OSM - 2016-2018")
plt.show()

TypeError: __init__() got an unexpected keyword argument 'orient'

In [49]:
os.path.getsize("test.db")

34798592

# Literature



1. On Wrocław https://en.wikipedia.org/wiki/Wroc%C5%82aw